<a href="https://colab.research.google.com/github/RGivisiez/Adv_ML/blob/clean/XGboost_and_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Importing

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from itertools import product

import xgboost as xgb
from xgboost import XGBRegressor

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout
from keras.models import Sequential

Using TensorFlow backend.


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [2]:
path = '~/Git_repo/Adv_ML/data'

# Loading data

In [5]:
train = pd.read_csv(path + '/sales_train.csv.gz')
test = pd.read_csv(path + '/test.csv.gz')
items = pd.read_csv(path + '/items.csv')
items_cat = pd.read_csv(path + '/item_categories.csv')
shops = pd.read_csv(path + '/shops.csv')

#print('Treino \n', train.head(), '\n')
#print('Itens \n', items.head(), '\n')
#print('Itens Cat \n', items_cat.head(), '\n')
#print('Shops \n', shops.head(), '\n')

In [4]:
print('Teste \n', test.tail() )

Teste 
             ID  shop_id  item_id
214195  214195       45    18454
214196  214196       45    16188
214197  214197       45    15757
214198  214198       45    19648
214199  214199       45      969


# Creating new features

In [6]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops.drop(['shop_name', 'city'], axis='columns', inplace=True)
#shops.head()

In [7]:
items_cat['split'] = items_cat['item_category_name'].str.split('-')
items_cat['type'] = items_cat['split'].map(lambda x: x[0].strip())
items_cat['type_code'] = LabelEncoder().fit_transform(items_cat['type'])

items_cat['subtype'] = items_cat['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
items_cat['subtype_code'] = LabelEncoder().fit_transform(items_cat['subtype'])

items_cat.drop(['item_category_name', 'split', 'type', 'subtype'], axis='columns', inplace=True)
#items_cat.head()

In [8]:
items.drop(['item_name'], axis=1, inplace=True)
#items.head()

In [9]:
train.drop(['date'], axis='columns', inplace=True)
#train.head()

In [46]:
train['revenue'] = train['item_price'] * train['item_cnt_day']
train.drop(['item_price'], axis=1, inplace=True)

# Train_full é só pra fazer EDA

In [ ]:
train_full = pd.merge(train, shops, on='shop_id', how='left')
train_full = pd.merge(train_full, items, on='item_id', how='left')
train_full = pd.merge(train_full, items_cat, on='item_category_id', how='left')
#train_full.head()

In [ ]:
# Liner correlation between features.
correlation = train_full.corr()
plt.figure(figsize=(15,15))
sns.heatmap(correlation, vmax=1, square=True,annot=True)
plt.show()

In [ ]:
print('Test:')
print('- Número de itens únicos:', test['item_id'].nunique())
print('- Número de shoppings únicos:', test['shop_id'].nunique())
print('\n Train_full:')
print('- Número de itens únicos:', train_full['item_id'].nunique())
print('- Número de shoppings únicos:', train_full['shop_id'].nunique())

In [ ]:
train_full.describe()

In [ ]:
test.describe()

In [ ]:
plt.hist(train_full['item_price'], bins = 100)
plt.title('Hist - item_price')
plt.xlabel('Item Price')
plt.show()

In [ ]:
plt.hist(train_full['item_price'][train_full.item_price < 8000], bins = 100)
plt.title('Hist - item_price')
plt.xlabel('Item Price')
plt.show()

In [ ]:
x = train_full['item_price'][train_full.item_price < 8000].values
x = np.log(np.abs(x))
plt.hist(x, bins = 30)
plt.title('Hist - item_price')
plt.xlabel('Log(Item Price)')
plt.show()

In [ ]:
train_full = train_full[train_full.item_price < 100000]
train_full = train_full[train_full.item_cnt_day < 1001]

In [ ]:
train_full.describe()

# Faz alterações no train e test

In [10]:
# Idk with this is true
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [11]:
train = train[train.item_price < 100000]
train = train[train.item_cnt_day < 1001]

# Months features

In [58]:
df_month = train.groupby(by=['date_block_num', 'shop_id', 'item_id'], axis='index').sum()
df_month.rename(columns={'item_cnt_day':'item_cnt_month', 'revenue':'revenue_month'}, inplace=True)
df_month.reset_index(inplace=True)
#df_month.head()

# Train and test more equals

In [ ]:
n_months = train['date_block_num'].nunique()
aux = []
for i in range(n_months):
    df = train[train.date_block_num == i]
    shop_uniq, item_uniq = df.shop_id.unique(), df.item_id.unique()

    a = list(product([i], shop_uniq, item_uniq))
    a = np.array(a)
    aux.append(a)
aux = pd.DataFrame(np.vstack(aux), columns=['date_block_num', 'shop_id', 'item_id'])
aux.sort_values(['date_block_num', 'shop_id', 'item_id'], inplace=True)

In [ ]:
df_month = pd.merge(aux, df_month, on=['date_block_num', 'shop_id', 'item_id'], how='left').fillna(0)
del aux

-----------------

In [59]:
df_month['item_cnt_month'] = df_month['item_cnt_month'].clip(0, 20)

# Adiciona o teste ao treino

In [60]:
test['date_block_num'] = 34
test.drop(['ID'], axis='columns', inplace=True)
#test.head()

KeyError: "['ID'] not found in axis"

In [61]:
df_month = pd.concat([df_month, test], ignore_index=True, sort=False, keys=['date_block_num', 'shop_id', 'item_id']).fillna(0)
#df_month.tail()

In [62]:
df_month = pd.merge(df_month, shops, on=['shop_id'], how='left')
df_month = pd.merge(df_month, items, on=['item_id'], how='left')
df_month = pd.merge(df_month, items_cat, on=['item_category_id'], how='left')

In [ ]:
df_month.head()

In [64]:
df_month['date_block_num'] = df_month['date_block_num'].astype('int8')
df_month['shop_id'] = df_month['shop_id'].astype('int16')
df_month['item_id'] = df_month['item_id'].astype('int16')
df_month['item_cnt_month'] = df_month['item_cnt_month'].astype('int16')
df_month['item_category_id'] = df_month['item_category_id'].astype('int16')
df_month['revenue_month'] = df_month['revenue_month'].astype('float32')
df_month['city_code'] = df_month['city_code'].astype('int16')
df_month['type_code'] = df_month['type_code'].astype('int16')
df_month['subtype_code'] = df_month['subtype_code'].astype('int16')

In [65]:
df_month.describe()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,city_code,item_category_id,type_code,subtype_code
count,1.823323e+06,1.823323e+06,1.823323e+06,1.823323e+06,1.823323e+06,1.823323e+06,1.823323e+06,1.823323e+06,1.823323e+06
mean,1.693625e+01,3.293353e+01,1.072075e+04,1.785172e+00,1.863968e+03,1.602764e+01,4.210200e+01,1.036932e+01,1.850362e+01
std,1.091414e+01,1.659080e+01,6.241452e+03,2.507862e+00,1.440297e+04,7.734857e+00,1.644211e+01,3.348347e+00,1.989422e+01
min,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,-3.378700e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.000000e+00,2.100000e+01,5.070000e+03,1.000000e+00,1.980000e+02,1.300000e+01,3.000000e+01,8.000000e+00,2.000000e+00
50%,1.600000e+01,3.100000e+01,1.059100e+04,1.000000e+00,3.990000e+02,1.300000e+01,4.000000e+01,1.100000e+01,1.000000e+01
75%,2.600000e+01,4.800000e+01,1.606300e+04,2.000000e+00,1.199000e+03,2.300000e+01,5.500000e+01,1.300000e+01,3.200000e+01
max,3.400000e+01,5.900000e+01,2.216900e+04,2.000000e+01,6.264230e+06,3.000000e+01,8.300000e+01,1.900000e+01,6.400000e+01


# PCA

-https://www.kaggle.com/arthurtok/interactive-intro-to-dimensionality-reduction
- http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html

In [ ]:
col = ['city_code', 'item_category_id', 'type_code']

In [ ]:
scaler = StandardScaler().fit(df_month[col]) # salva a média e std de cada coluna, para poder ser aplicado no set de treino.
X = scaler.transform(df_month[col])
print('\nMean:', scaler.mean_, '\n\nStd:', scaler.scale_)

In [ ]:
#@title Dataframe centrado em zero com variância 1.
pd.DataFrame(X, columns=df_month.columns).describe()

In [ ]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents)

In [ ]:
plt.plot(pca.explained_variance_ratio_, marker='o')
plt.ylabel('Importance')
plt.xlabel('Component number')
plt.show()

In [ ]:
df_month.drop(col, axis=1, inplace=True)

In [ ]:
df_month['PCA1'] = principalDf[0]
df_month['PCA2'] = principalDf[1]

# Lagged features

In [ ]:
df_lag3 = df_month[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'revenue_month']].copy()
df_lag3.rename(columns={'item_cnt_month':'lag3_item_cnt_month', 'revenue_month':'lag3_revenue_month'}, inplace=True)
df_lag3['date_block_num'] += 3
df_month = pd.merge(df_month, df_lag3, on=['date_block_num', 'shop_id', 'item_id'], how='left').fillna(0)
del df_lag3

In [ ]:
df_lag6 = df_month[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'revenue_month']].copy()
df_lag6.rename(columns={'item_cnt_month':'lag6_item_cnt_month', 'revenue_month':'lag6_revenue_month'}, inplace=True)
df_lag6['date_block_num'] += 6
df_month = pd.merge(df_month, df_lag6, on=['date_block_num', 'shop_id', 'item_id'], how='left').fillna(0)
del df_lag6

In [ ]:
df_lag9 = df_month[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'revenue_month']].copy()
df_lag9.rename(columns={'item_cnt_month':'lag9_item_cnt_month', 'revenue_month':'lag9_revenue_month'}, inplace=True)
df_lag9['date_block_num'] += 9
df_month = pd.merge(df_month, df_lag9, on=['date_block_num', 'shop_id', 'item_id'], how='left').fillna(0)
del df_lag9

In [ ]:
df_lag12 = df_month[['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'revenue_month']].copy()
df_lag12.rename(columns={'item_cnt_month':'lag12_item_cnt_month', 'revenue_month':'lag12_revenue_month'}, inplace=True)
df_lag12['date_block_num'] += 12
df_month = pd.merge(df_month, df_lag12, on=['date_block_num', 'shop_id', 'item_id'], how='left').fillna(0)
del df_lag12

In [ ]:
df_month['lag3_item_cnt_month'] = df_month['lag3_item_cnt_month'].astype('float32')
df_month['lag6_item_cnt_month'] = df_month['lag6_item_cnt_month'].astype('float32')
df_month['lag9_item_cnt_month'] = df_month['lag9_item_cnt_month'].astype('float32')
df_month['lag12_item_cnt_month'] = df_month['lag12_item_cnt_month'].astype('float32')

df_month['lag3_revenue_month'] = df_month['lag3_revenue_month'].astype('float32')
df_month['lag6_revenue_month'] = df_month['lag6_revenue_month'].astype('float32')
df_month['lag9_revenue_month'] = df_month['lag9_revenue_month'].astype('float32')
df_month['lag12_revenue_month'] = df_month['lag12_revenue_month'].astype('float32')

In [ ]:
df_month.info()

In [ ]:
df_month.head()

In [ ]:
col2 = ['item_price_month', 'revenue_month', 'lag3_item_cnt_month', 'lag3_revenue_month', 'lag6_item_cnt_month', 'lag6_revenue_month', 'lag9_item_cnt_month', 'lag9_revenue_month', 'lag12_item_cnt_month', 'lag12_revenue_month']

In [ ]:
scaler = StandardScaler().fit(df_month[col2]) # salva a média e std de cada coluna, para poder ser aplicado no set de treino.
X = scaler.transform(df_month[col2])
df_month[col2] = pd.DataFrame(X, columns=col2)
df_month.head()

# XGboost

In [ ]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=100,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8,
    nthreads = 6,
    seed = 42,
    eta=0.3)

X_train = df_month[df_month.date_block_num < 33].drop(['item_cnt_month'], axis='columns')
Y_train = df_month[df_month.date_block_num < 33]['item_cnt_month']

X_valid = df_month[df_month.date_block_num == 33].drop(['item_cnt_month'], axis='columns')
Y_valid = df_month[df_month.date_block_num == 33]['item_cnt_month']

X_test = df_month[df_month.date_block_num == 34].drop(['item_cnt_month'], axis=1)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 5)

In [ ]:
xgb.plot_importance(model, importance_type='weight')
plt.show()

In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

In [ ]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

# RNN

- Samples: One sequence is one sample. A batch is comprised of one or more samples.
- Time Steps: One time step is one point of observation in the sample.
- Features: One feature is one observation at a time step.

In [251]:
X_train = df_month.pivot_table(index=['shop_id', 'item_id', 'item_category_id'],
                               columns=['date_block_num'], values=['item_cnt_month'],
                               fill_value=0)
#X_train = df_month.pivot_table(index=['shop_id', 'city_code', 'item_id', 'type_code',
#                                      'subtype_code', 'revenue_month', 'item_category_id'],
#                               columns=['date_block_num'], values=['item_cnt_month'],
#                               fill_value=0)

X_train.head()

item_cnt_month                             \
date_block_num                               0  1  2  3  4  5  6  7  8  9    
shop_id item_id item_category_id                                             
2       27      19                            1  0  0  0  0  0  0  0  0  0   
        30      40                            0  0  1  0  0  1  0  0  0  0   
        31      37                            0  4  1  1  0  0  0  0  0  0   
        32      40                            0  0  0  0  0  0  0  0  0  0   
        33      37                            1  0  0  0  0  0  0  0  0  0   

                                 ...                                
date_block_num                   ... 25 26 27 28 29 30 31 32 33 34  
shop_id item_id item_category_id ...                                
2       27      19               ...  0  0  0  0  0  0  0  0  0  0  
        30      40               ...  0  0  0  0  0  0  0  0  0  0  
        31      37               ...  0  0  0  0  0  0  0  0  1  0  
        32      40               ...  0  1  0  0  0  0  1  0  0  0  
        33      37               ...  1  0  1  1  0  1  0  1  0  0  

[5 rows x 35 columns]

In [252]:
X_train.columns = X_train.columns.droplevel(0)
X_train.reset_index(inplace=True)

In [253]:
X_train.drop(['item_category_id'], axis=1, inplace=True)

In [217]:
#X_train.drop(['shop_id', 'item_id'], axis=1, inplace=True)

In [254]:
X_train.columns = ['shop_id', 'item_id'] + [str(x) for x in range(35)]

In [263]:
Y_train = X_train.loc[:, ['shop_id', 'item_id', '33']]
X_train = X_train.drop(['33', '34'], axis=1)

In [266]:
n_samples, time_steps = X_train.shape
features = 1
print(n_samples, time_steps, features)

521637 35 1


In [271]:
X_train = X_train.values.reshape((n_samples, time_steps, features))

In [274]:
model = Sequential()
model.add(LSTM(units=32, input_shape=(time_steps, features)))
model.add(Dense(1))
model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 4,451
Trainable params: 4,451
Non-trainable params: 0
_________________________________________________________________


In [275]:
model.fit(X_train, Y_train, batch_size = 4096, epochs = 10)

Epoch 1/10
521637/521637 [==============================] - 8s 14us/step - loss: 56008323.1046 - mean_squared_error: 56008323.1046
Epoch 2/10
521637/521637 [==============================] - 7s 13us/step - loss: 55946910.2192 - mean_squared_error: 55946910.2192
Epoch 3/10
521637/521637 [==============================] - 7s 14us/step - loss: 55909223.8778 - mean_squared_error: 55909223.8778
Epoch 4/10
521637/521637 [==============================] - 7s 14us/step - loss: 55874385.3537 - mean_squared_error: 55874385.3537
Epoch 5/10
521637/521637 [==============================] - 7s 14us/step - loss: 55840535.4220 - mean_squared_error: 55840535.4220
Epoch 6/10
521637/521637 [==============================] - 7s 14us/step - loss: 55807218.8223 - mean_squared_error: 55807218.8223
Epoch 7/10
521637/521637 [==============================] - 7s 14us/step - loss: 55774237.7000 - mean_squared_error: 55774237.7000
Epoch 8/10
521637/521637 [==============================] - 7s 14us/step - loss: 55

In [186]:
#X_train.reset_index().iloc[0]

In [130]:
#X_train.columns = X_train.columns.droplevel(0)
#X_train.T.reset_index()